In [11]:
import pennylane as qml
import numpy as np
from pennylane import numpy as pnp
from matplotlib import pyplot as plt
from pennylane.operation import Operation, AnyWires
import os
import pandas as pd
import scipy.special as sp
import math
from scipy.stats import uniform_direction
from scipy.linalg import logm, svd

num_qubits = 5

# Initialize the device
dev = qml.device("default.qubit", wires=num_qubits)
dev2 = qml.device("default.qubit", wires=1)

In [12]:
# Construct the Hamiltonian terms
Hamiltonian_terms = []

# Interaction terms: XiX(i+1) + YiY(i+1) + ZiZ(i+1)
for i in range(num_qubits):
    Hamiltonian_terms.append(1.0 * (qml.PauliX(i) @ qml.PauliX((i+1)%num_qubits)) +
                                    (qml.PauliY(i) @ qml.PauliY((i+1)%num_qubits)) 
                                    + (qml.PauliZ(i) @ qml.PauliZ((i+1)%num_qubits)))

# Magnetic field terms: hZi
for i in range(num_qubits):
    Hamiltonian_terms.append(1.0 * qml.PauliZ(i))

# Define the Hamiltonian
Hamiltonian_operator = qml.Hamiltonian(coeffs=[1] * len(Hamiltonian_terms), observables=Hamiltonian_terms)

In [13]:
Hamiltonian = qml.Hamiltonian(coeffs=[1] * len(Hamiltonian_terms), observables=Hamiltonian_terms)

In [14]:


class fqs_op(qml.operation.Operation):
    num_params = 1
    num_wires = qml.operation.AnyWires
    par_domain = "R"

    @staticmethod
    def compute_matrix(axis):  # theta is the rotation angle
        """Custom operation for free-axis rotation"""
        q0, q1, q2, q3 = axis
        H = qml.sum(q0 * qml.Identity(AnyWires),  - 1j * q1 * qml.X(AnyWires), -1j * q2 * qml.Y(AnyWires), -1j * q3 * qml.Z(AnyWires))
        
        return H.matrix()

In [15]:

def entangling_layer_ladderZ(num_qubits):
    m = 0
    n = 1
    while m+1 < num_qubits:
        qml.CZ(wires=[m,m+1])
        m+=2
    
    while n+1 < num_qubits:
        qml.CZ(wires=[n,n+1])
        n+=2


@qml.qnode(dev)
def circuit(n_vectors, num_layers):
    """Parameterized quantum circuit with free-axis rotations"""
    
    for j in range(num_layers):
        for k in range(num_qubits):
            fqs_op(n_vectors[k + num_qubits * j], wires = k)
    
        entangling_layer_ladderZ(num_qubits)

    return qml.expval(Hamiltonian)

@qml.qnode(dev)
def circuit_state(n_vectors, num_layers, d, gate_type):
    """Parameterized quantum circuit with free-axis rotations"""
    
    ind = 0

    for j in range(num_layers):
    
        for k in range(num_qubits):

            if ind == d:
                if gate_type == "X":
                    fqs_op([0,1,0,0], wires=k)

                elif gate_type == "Y":
                    fqs_op([0,0,1,0], wires=k)

                elif gate_type == "Z":
                    fqs_op([0,0,0,1], wires=k)

                elif gate_type == "XY":
                    fqs_op([0, 1/np.sqrt(2), 1/np.sqrt(2), 0], wires=k)
   
                elif gate_type == "XZ":
                    fqs_op([0, 1/np.sqrt(2), 0, 1/np.sqrt(2)], wires=k)

                elif gate_type == "YZ":
                    fqs_op([0, 0, 1/np.sqrt(2), 1/np.sqrt(2)], wires=k)
                
                elif gate_type == "I":
                    fqs_op([1,0,0,0], wires=k)

                elif gate_type == "I_X":
                    fqs_op([1/np.sqrt(2), 1/np.sqrt(2), 0, 0], wires=k)

                elif gate_type == "I_Y":
                    fqs_op([1/np.sqrt(2),0, 1/np.sqrt(2), 0], wires=k)
                
                elif gate_type == "I_Z":
                    fqs_op([1/np.sqrt(2),0,0, 1/np.sqrt(2)], wires=k)

            else:
                fqs_op(n_vectors[k + num_qubits * j], wires = k)

            ind += 1
    
        entangling_layer_ladderZ(num_qubits)

    
    return qml.expval(Hamiltonian)


In [16]:

def compute_fqs_matrix(n_vectors, num_layers, d):
    """Compute the fqs matrix for a specific gate d"""

    rx = circuit_state(n_vectors, num_layers, d, gate_type="X")
    ry = circuit_state(n_vectors, num_layers, d, gate_type="Y")
    rz = circuit_state(n_vectors, num_layers, d, gate_type="Z")
    
    rxy = circuit_state(n_vectors, num_layers, d, gate_type="XY")
    rxz = circuit_state(n_vectors, num_layers, d, gate_type="XZ")
    ryz = circuit_state(n_vectors, num_layers, d, gate_type="YZ")
    
    Id = circuit_state(n_vectors, num_layers, d, gate_type="I")
    Id_x = circuit_state(n_vectors, num_layers, d, gate_type="I_X")
    Id_y = circuit_state(n_vectors, num_layers, d, gate_type="I_Y")
    Id_z = circuit_state(n_vectors, num_layers, d, gate_type="I_Z")
    
    #print(Id_z)


    matrix = [[Id             , Id_x-rx/2-Id/2, Id_y-ry/2-Id/2, Id_z-rz/2-Id/2],
                [Id_x-rx/2-Id/2,  rx            , (2*rxy-rx-ry)/2, (2*rxz-rx-rz)/2],
                [Id_y-ry/2-Id/2, (2*rxy-rx-ry)/2,  ry            , (2*ryz-ry-rz)/2],
                [Id_z-rz/2-Id/2, (2*rxz-rx-rz)/2, (2*ryz-ry-rz)/2,  rz            ]]

    return matrix



In [17]:
Id = np.matrix([[1,0],
               [0,1]])

X = np.matrix([[0,1],
               [1,0]])

Y = np.matrix([[0,-1j],
               [1j,0]])

Z = np.matrix([[1,0],
               [0,-1]])

In [ ]:

def quaternion_to_unitary(q):
    U = q[0] * Id - 1j * q[1] * X - 1j * q[2] * Y - 1j * q[3] * Z
    return U

def is_unitary(U):
    return np.allclose(U.conj().T @ U, np.eye(U.shape[0]))

def spectral_norm_distance(U, V):
    diff = U - V
    return svd(diff, compute_uv=False)[0]  # Largest singular value

def frobenius_distance(U, V):
    diff = U - V
    return np.linalg.norm(diff, 'fro')

def bloch_dist(U, V):
    inner_product = np.trace(U.conj().T @ V)
    #angle = np.arccos(np.clip(np.real(inner_product) / 2, -1.0, 1.0))
    dist = np.sqrt(2.0 - np.abs(inner_product))
    
    return dist


In [19]:


def free_quaternion_selection(n_vectors, num_layers, iters, freeze_threshold, freeze_iters_k):
    """Implement the Fraxis algorithm"""

    num_gates = len(n_vectors)

    all_vals = []

    freeze_counters = np.zeros(len(n_vectors))    
    
    gate_opts_tresh = num_qubits * num_layers * iters 
    gate_opts = 0
    
    while True:

        if gate_opts > gate_opts_tresh:
            break

        for d in range(num_gates):
            
            if gate_opts > gate_opts_tresh:
                break
            
            if freeze_counters[d] > 0:
                freeze_counters[d] = freeze_counters[d] - 1
                #print(d)
                continue

            prev_q = np.array(n_vectors[d].copy())

            current_val = circuit(n_vectors, num_layers)

            # Use this for ideal simulator
            if len(all_vals) > 0:
                if current_val > all_vals[-1] or current_val < min(Hamiltonian.eigvals()):
                    all_vals.append(all_vals[-1])
                else:
                    all_vals.append(current_val)
            else:
                all_vals.append(current_val)
            
            fqs_matrix = compute_fqs_matrix(n_vectors, num_layers, d)        

            eigVal, eigVec = np.linalg.eig(fqs_matrix)
            eigVec = np.transpose(eigVec)

            sid = np.argmin(eigVal)
            expected_val = np.amin(eigVal)
            
            if expected_val < current_val:
                n_vectors[d]  = eigVec[sid]

            current_q = np.array(n_vectors[d].copy())

            prev_unitary =  quaternion_to_unitary(prev_q)
            current_unitary = quaternion_to_unitary(current_q)

            bloch_dist_normalized = bloch_dist(prev_unitary, current_unitary) / np.sqrt(2.0)

            if (bloch_dist_normalized < freeze_threshold):
                #freeze_counters[d] = freeze_iters_k[d]
                freeze_iters_k[d] += 1
                #print("Freeze d,", d)
                
            gate_opts += 1

    return n_vectors, all_vals, freeze_iters_k 



In [20]:
# Initialize parameters and run optimization
layers = [3]
iters = 30

trials = 50

hs_vals = [0.01, 0.005, 0.025, 0.001]
uniform_sphere_dist = uniform_direction(4)

for hs_val in hs_vals:
    # freeze threshold as the angle 
    freeze_threshold = hs_val
    
    for num_layers in layers:
        for trial in range(trials):
            print("trials", trial+1)
            num_gates = num_qubits * num_layers
            freeze_iters = np.ones(num_gates)

            n_vectors = uniform_sphere_dist.rvs(num_gates)
            
            optimal_n_vectors, opt_vals, freeze_iters = free_quaternion_selection(n_vectors, num_layers, iters, freeze_threshold, freeze_iters)
            #print(freeze_iters_k)
            
            file2 = f"BlochDist_gateData_1DHeisenberg_{num_qubits}Q_FQS_GateFreeze_Val{hs_val}_FreezeIterInc_{iters}cycles_{num_layers}layers_{trials}trials_A.xlsx"      
            
            if not os.path.exists(file2):
                df2 = pd.DataFrame()
                df2.to_excel(file2)

            df2 = pd.read_excel(file2)

            if len(df2.columns) < trials:
                
                df2[f"col{len(df2.columns)}"] = pd.Series(freeze_iters)
                df2.to_excel(file2,index = False)
            else:
                break
                



trials 1
trials 2


C:\Users\joonp\AppData\Local\Temp\ipykernel_3400\1836410383.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  n_vectors[d]  = eigVec[sid]


trials 3
trials 4
trials 5


C:\Users\joonp\AppData\Local\Temp\ipykernel_3400\1836410383.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  n_vectors[d]  = eigVec[sid]


trials 6
trials 7
trials 1
trials 2
trials 3


...

trials 8
trials 9
trials 10
trials 11
trials 12
trials 13
trials 14
trials 15
trials 16
trials 17


C:\Users\joonp\AppData\Local\Temp\ipykernel_3400\1836410383.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  n_vectors[d]  = eigVec[sid]


trials 18
trials 19
trials 20
trials 21


C:\Users\joonp\AppData\Local\Temp\ipykernel_3400\1836410383.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  n_vectors[d]  = eigVec[sid]


trials 22
trials 23
trials 24
trials 25
trials 26
trials 27
trials 28
trials 29
trials 30


In [21]:
print(qml.draw(circuit)(n_vectors, num_layers))


0: ──fqs_op(M0)─╭●──fqs_op(M5)─────────────╭●──fqs_op(M10)──────────────╭●────┤ ╭<𝓗>
1: ──fqs_op(M1)─╰Z─╭●───────────fqs_op(M6)─╰Z─╭●────────────fqs_op(M11)─╰Z─╭●─┤ ├<𝓗>
2: ──fqs_op(M2)─╭●─╰Z───────────fqs_op(M7)─╭●─╰Z────────────fqs_op(M12)─╭●─╰Z─┤ ├<𝓗>
3: ──fqs_op(M3)─╰Z─╭●───────────fqs_op(M8)─╰Z─╭●────────────fqs_op(M13)─╰Z─╭●─┤ ├<𝓗>
4: ──fqs_op(M4)────╰Z───────────fqs_op(M9)────╰Z────────────fqs_op(M14)────╰Z─┤ ╰<𝓗>

M0 = 
[0.05773767 0.12080961 0.64872449 0.74915148]
M1 = 
[ 0.29691982 -0.25679618 -0.35090674  0.85015222]
M2 = 
[-0.36554494 -0.22176514  0.51311058 -0.74425443]
M3 = 
[-0.12525196 -0.23169869  0.96152093  0.07813559]
M4 = 
[-0.78372985 -0.15635386  0.28773197  0.52776065]
M5 = 
[-0.0828286  -0.44845438 -0.88342519 -0.1076477 ]
M6 = 
[-0.33028116  0.69996453  0.21030917 -0.59727219]
M7 = 
[-0.33118813  0.8422957  -0.35944844  0.2272646 ]
M8 = 
[-0.68825569 -0.04438652 -0.71783967  0.09507968]
M9 = 
[ 0.75419632 -0.28132784  0.10061844 -0.58473796]
M10 = 
[-0.1495095